# Projet IA - 4BiM - 2021

#### Auteurs : Aurélie Fischer et Nicolas Mendiboure 
#### Enseignants : Sergio Peignier et Chistophe Rigotti

In [ ]:
%matplotlib inline 

In [ ]:
import scipy
import numpy
import math
import sklearn
import pandas as pds
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch

#import sergio_peignier as sp
#import chistophe_rigotti as cr

# 1. Lecture des données:

In [ ]:
sparse_df = pds.read_csv("./datas/Data-IA-World-Development-Indicator.txt", sep="\t", header=0)

In [ ]:
sparse_df.shape

In [ ]:
#sparse_df.head()
#sparse_df.tail()

In [ ]:
# Quelques statistiques :
sparse_df.describe() 

# 2. Création des classes (continents) :

In [ ]:
import pycountry_convert as pc

In [ ]:
country_code = sparse_df[["Country Name", "Country Code"]]

In [ ]:
country_code_np = country_code["Country Code"].values

In [ ]:
def country_to_continent(country_code):
    res = []
    for code3 in country_code :
        if (code3 == "CHI"):
            code2 = "GB"
        elif (code3 == "XKX"):
            code2 = "XK"
        else:
            code2 = pc.country_alpha3_to_country_alpha2(code3)
            
        
        if (code2 == "TL"):
            country_continent_code = "AS"
        elif (code2 == "SX"):
            country_continent_code = "SA"
        else:
            country_continent_code = pc.country_alpha2_to_continent_code(code2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        res.append(country_continent_name)
    return res

In [ ]:
continents = country_to_continent(country_code_np)

In [ ]:
sparse_df["Continents"] = continents

# 3. Selection et filtrage des données :

## 3.1 Selection des attributs (colonnes) :

Premiers apperçus des données du jeu (décommenter pour afficher) :

In [ ]:
#Fonction qui supprime une colonne (attribut) si le nb de NaN >= limit

def delete_NaN_col(df, limit):
    NaN_col = df.isna().sum()
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[1]):
        if NaN_col[i] >= limit:
            df = df.drop(columns = tmp.columns[i])
            
    return(df)

In [ ]:
sparse_df2 = delete_NaN_col(sparse_df, 40)

In [ ]:
# On enlèvre les colonnes 'Time' et 'Time Code' qui ne nous interessent pas ici
def remove_useless_col(df):
    df = df.drop(columns = ["Time", "Time Code", "Country Code"])
    return (df)

In [ ]:
sparse_df2 = remove_useless_col(sparse_df2)

## 3.2 Selection des objets (lignes) :

In [ ]:
sparse_df2.isna().sum()

In [ ]:
sparse_df2.isna().sum(axis=1)

In [ ]:
#Fonction qui supprime une ligne (object) si le nb de NaN >= limit

def delete_NaN_row(df, limit):
    NaN_row = df.isna().sum(axis=1)
    tmp = df.copy(deep=True) #temporary df
    
    for i in range(df.shape[0]):
        if NaN_row[i] >= limit:
            df = df.drop([i], axis = 0)
            
    return(df)

In [ ]:
sparse_df3 = delete_NaN_row(sparse_df2, 1)

In [ ]:
sparse_df3

In [ ]:
#On remplace les indexes des ligne {0, 1, 2, ..n} par les noms de pays
#Puis on supprime la colonne "Countru Name" pour n'avoir plus que des valeurs numériques
sparse_df3.index = sparse_df3['Country Name']
sparse_df3 = sparse_df3.drop(columns = ['Country Name'])

In [ ]:
sparse_df3.head()

## 3.3 Études des correlations :

In [ ]:
corr_df = sparse_df3.corr()
corr_df[:5]

In [ ]:
# Représentation graphique des correlations
plt.figure(figsize= (15, 12))
sns.heatmap(corr_df,annot=True)
plt.show()

In [ ]:
sns.clustermap(corr_df,
               figsize= (16, 12),
               annot=True,
               dendrogram_ratio=(0.1, 0.2),
               row_cluster=False)
plt.show()

In [ ]:
def delete_corr(df, limit):
    #On construit notre matrice de correlation en valeur absolue
    corr_matrix = df.corr().abs()
    
    #Triangle supérieur de la matrice de corrélation : 
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > limit)]
    df = df.drop(df[to_drop], axis=1)
    return(df)

In [ ]:
df = delete_corr(sparse_df3, 0.55)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize= (15, 12))
sns.heatmap(df.corr().abs(), annot = True)
plt.show()

In [ ]:
#On ne sait pas vraiment à quoi correspond cet attribut
#De plus il faudrait normaliser ses valeurs, sauf que nous ne connaissons aucun référentiel pour faire cela
df = df.drop(columns=["Net domestic credit (current LCU) [FM.AST.DOMS.CN]"]) 

In [ ]:
# On renomme les labels des colonnes pour plus de fluidité
df = df.rename(columns={"Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]" : "Clean fuels and technologies for cooking acces (% pop)"})
df = df.rename(columns={"Aquaculture production (metric tons) [ER.FSH.AQUA.MT]" : "Aquaculture prod (metric tons)"})
df = df.rename(columns={"Compulsory education, duration (years) [SE.COM.DURS]" : "Compulsory education duration (years)"})
df = df.rename(columns={"Death rate, crude (per 1,000 people) [SP.DYN.CDRT.IN]" : "Death rate (per 10e3 )"})
df = df.rename(columns={"Incidence of tuberculosis (per 10e5 people) [SH.TBS.INCD]" : "Incidence of tuberculosis (per 100 000)"})
df = df.rename(columns={"Number of infant deaths [SH.DTH.IMRT]" : "Number of infant deaths"})
df = df.rename(columns={"Preprimary education, duration (years) [SE.PRE.DURS]" : "Preprimary education duration (years)"})

In [ ]:
df.head()

## 3.4 Normalisation relative (aux surfaces et poplations par pays) :

In [ ]:
def remove_country(df, df_ref):
    tmp = df.copy(deep = True)
    for i, country in  enumerate( tmp["Country Name"]):
        if country not in df_ref.index.values:
            df = df.drop(tmp.index[i], axis = 0)
    return (df)

In [ ]:
population_size = pds.read_csv("./datas/Popula-schtroumpf/Population-size-per-country.txt", sep="\t", header=0)
population_size = remove_useless_col(population_size)
population_size = delete_NaN_row(population_size, 1)
population_size = remove_country(population_size, df)

In [ ]:
surfaces = pds.read_csv("./datas/Surfa-schtroumpf/Country-surfaces.txt", sep="\t", header=0)
surfaces = remove_useless_col(surfaces)
surfaces = delete_NaN_row(surfaces, 1)
surfaces = remove_country(surfaces, df)

In [ ]:
print (len(population_size) == len(df), len(surfaces)== len(df))

In [ ]:
df2 = df.copy(deep = True)
print("Avant :", "\n")
df2.head()

In [ ]:
df2.iloc[:, 1] =  df2.iloc[:, 1].values/surfaces.iloc[:, 1].values
df2.iloc[:, 5] =  df2.iloc[:, 5].values/population_size.iloc[:, 1].values
print("Après :", "\n")
df2.head()

## 3.5 Normalisation par centrage - réduction :

\begin{equation*}
    X = \frac{X - \mu}{\sigma}
\end{equation*}

In [ ]:
sns.pairplot(data=df2)
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
sns.boxplot(data = df2)
plt.show()

#On voit que les ordres de grandeur de nos données ainsi que leur variances ne sont pas du tout homogènes.
#On va donc les normaliser (centrer - réduire) pour corriger ce défaut  

In [ ]:
# Fonction qui centre et réduit une colonne de données pour les normaliser
def normalizer (data):
    # Quelques statistiques :
    mean = np.mean(data)
    var = np.var(data)
    sd = math.sqrt(var)
    
    norm = []
    for x in data:
        norm.append((x-mean)/sd)
    
    return (norm)

In [ ]:
def norm_whole_df (df):
    normed_df = df.copy(deep = True)
    for i in df.columns:
        normed_df[i] = normalizer(df[i])
        
    return (normed_df)

In [ ]:
normed_df = norm_whole_df(df2.drop(columns = "Continents" ))
normed_df["Continents"] = df2["Continents"]

In [ ]:
normed_df.head()

In [ ]:
normed_df.shape

In [ ]:
plt.figure(figsize=(12,15))
sns.boxplot(data = normed_df)
plt.show()

In [ ]:
from scipy import stats

In [ ]:
def search_outliers_row(df, limit):
    z_scores = stats.zscore(df.drop(columns = "Continents" )) #calcule le z-score de df
    abs_z_scores = np.abs(z_scores) # on passe en valeurs absolues
    filtered_outliers = (abs_z_scores < limit).all(axis=1) # on ne garde que les lignes avec des valeurs > limit
    new_df = df[filtered_outliers] #nouveau df
    return (new_df)

In [ ]:
df3 = search_outliers_row(normed_df, 3)

In [ ]:
df3.shape

In [ ]:
plt.figure(figsize=(12,15))
sns.boxplot(data = df3)
plt.show()

In [ ]:
#Pairplot après standardisation
sns.pairplot(data=df3)
plt.show()

On refait des statistiques basiques sur notre nouveau jeu de données df3. On voit bie nque notre moyenne est centrée sur 0 et que notre écart type varie autour de 1 pour chaque attribut.

In [ ]:
df_stats = df3.describe()
df_stats = df_stats.drop('count',axis=0)
sns.heatmap(df_stats,annot=True,cmap='coolwarm')
plt.show()

# 4. Clustering :

## 4.1 K-means 

### 4.1.1 Création des clusters :

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [ ]:
df4 = df3.drop(columns = "Continents" )
classes = df3.Continents

Nous allons utilisé le jeu de données fraichement filtré et normalisé df2 :

In [ ]:
df4.head()

In [ ]:
# Création d'un object KMeans :
km = KMeans(n_clusters=4, init='k-means++',n_init=10, random_state=50, max_iter=300,)
km.fit(df4)

In [ ]:
km_clusters = km.labels_

In [ ]:
# On stocke les centroides obtenus :
centroids = km.cluster_centers_

In [ ]:
predict = km.predict(df4)

In [ ]:
print(predict)

In [ ]:
# Using matplotlib.pyplot instead of seaborn.scatterplot to display the clusters.
plt.scatter(df4["Clean fuels and technologies for cooking acces (% pop)"], df4["Compulsory education duration (years)"], c=km_clusters)
plt.title('Data in Space', fontsize=14)
plt.xlabel("Clean fuels and technologies for cooking acces (% pop)",fontsize=14)
plt.ylabel("Compulsory education duration (years)",fontsize=14 )
plt.scatter(centroids[:, 0], centroids[:, 2], c='red',s=150, alpha=0.4)
plt.grid(True)

### 4.1.2 Mesures sur nos kmeans :

In [ ]:
from collections import Counter 
Counter(predict)

#### a) SSE :

In [ ]:
SSE=km.inertia_
print(SSE)

In [ ]:
#Courbe de la SSE en fonction du nombre de clusters
SSE_liste=[]
for i in range(2,10):
    km2=KMeans(n_clusters=i, init='k-means++',  n_init=1, random_state=50, max_iter=1000).fit(df4)
    SSE_liste.append(km2.inertia_)
    
#print(SSE_liste)

SSE_liste_random=[]
x=[]
for i in range(2,10):
    km3=KMeans(n_clusters=i, init='random',  n_init=1, random_state=50, max_iter=1000).fit(df4)
    SSE_liste_random.append(km3.inertia_)
    x.append(i)
    
#print(SSE_liste_random)
plt.figure(figsize = (14, 10))
plt.plot(x, SSE_liste,label="k-means++")
plt.plot(x, SSE_liste_random,label="Random")
plt.xlabel("Nombre de clusters")
plt.ylabel("SSE")
plt.legend()
plt.show()

Table de contingence avec les différents clusters vs les classes (ici continents) :

In [ ]:
km_crosstab = pds.crosstab(km_clusters,classes)
sns.heatmap(km_crosstab, annot=True)
plt.show()

#### b) Mesures d'entropie

Entropie des clusters (c) comparés aux différentes classes (j):

$-\sum{p(j|c) \times log(p(j|c)}$

$p(j|c)$ est la fréquence d'apparition de la classe j dans le cluster c.

In [ ]:
proba = km_crosstab.values/km_crosstab.values.sum(axis=1, keepdims=True) # divide each element of a row by the sum of the row
entropy = [stats.entropy(row, base=2) for row in proba]
print("entropy of each cluster: ", entropy)

#### c) Mesure de pureté :

Pour calculer la pureté, chaque cluster est attribué à la classe qui est la plus fréquente dans ce cluster, puis la précision de cette attribution est mesurée en comptant le nombre d'éléments correctement attribués. Il s'agit donc de mesurer la précision d'un cluster à ne contenir les objets que d'une seule classe.

En réalité il s'agit d'une terminologie dérivant de l'entropie.

$purete = \sum_{j=1}^{K} \frac{m_c}{m} \times purete(c)$

- $purete(c) = max(p(j,c))$;
- $p(j,c)$ est la probabilité qu'un membre du cluster $c$ appartienne à la classe $j$;
- $m_c$ est la taille du cluster $c$;
- $m$ est la taille totale du nombre d'éléments (points).

In [ ]:
def purity_score(classes, predictions):
    contingency_matrix = metrics.cluster.contingency_matrix(classes, predictions)
    return ( np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) )

In [ ]:
purity_score(classes, km_clusters)

<ins>Remarque :</ins> Une grande pureté est facile à atteindre lorsque le nombre de clusters formés devient important. En particulier, la pureté tend vers $1$ si le nombre de cluster tend à être égale aux nombre de points. Ainsi, nous ne pouvons pas utiliser la pureté pour faire un compromis entre la qualité du regroupement et le nombre de regroupements.

Une mesure qui nous permet de faire ce compromis est l'information mutuelle normalisée ou INM : 

#### d) Information mutuelle normalisée (IMN) :

$IMN(j,c) = \frac{2 \times I(j, c)}{E(j) + E(c)}$

- $j$ = label de la classe ;
- $c$ = label du cluster ;
- $E()$ = entropie ;
- $I(j, c)$ = information mutuelle entre j et c. $I(j, c) = E(j) - E(j|c)$, où $E(j|c)$ désigne une entropie conditionnelle.

In [ ]:
metrics.normalized_mutual_info_score(classes, km_clusters, average_method='max')

#### d) Coefficient de silhouette :

Le coefficient de silhouette pour un  point correspond à la différence entre sa distance moyenne avec les points du même cluster que lui (**cohésion**) et la distance moyenne avec les points des autres clusters voisins (**séparation**). Il est compris en $-1$ et $1$.

 - Si on a une différence négative, le point est en moyenne plus proche du groupe voisin que du sien ;
 - Si on a une différence positive, le point est en moyenne plus proche de son groupe que du groupe voisin.

In [ ]:
Silh = metrics.silhouette_score(df4.values, 
                         km_clusters, 
                         metric='euclidean', 
                         sample_size=None) 
print("Coefficient de silhouette pour nos 3 clusters : ",Silh)

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

Avec le code suivant nous pouvons observer les silhouettes de chaque cluster, pour un nombre total de  2, 3, 4, 5, 6 et 7 clusters

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(15,10))
for i in [2, 3, 4, 5, 6, 7]:
    
    km_simu = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
    q, mod = divmod(i, 2) # permet d'attribuer les axes pour les subplots 
    
    visualizer = SilhouetteVisualizer(km_simu, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(df4)

### 4.1.3 Stabilité de la convergence de nos k-means :

La convergence d'un algorithme K-Means peut être vue comme étant la stabilisation des centroids des clusters (les centroids ne bougent plus lors des itérations).

#### a) Calcul de la stabilité pour la convergence de notre K-means :

In [ ]:
avg_silhouette_coef = []
sse_list = []
k = 6
for i in range(600):
    km = KMeans(n_clusters=k, init='random', n_init=1)
    km.fit(df4)
    labels = km.predict(df4)
    sse_list.append(np.sqrt(km.inertia_))
    avg_silhouette_coef.append(metrics.silhouette_score(df4, labels,metric='euclidean'))

In [ ]:
plt.figure(figsize=(18, 10))

plt.subplot(2,2,1)
plt.hist(sse_list, edgecolor = "black")
plt.title("SSE", fontsize = 18)

plt.subplot(2,2,2)
plt.hist(avg_silhouette_coef, edgecolor = "black")
plt.title("Average silhouette coeff", fontsize = 18)

plt.subplot(2,2,3)
plt.boxplot(sse_list)

plt.subplot(2,2,4)
plt.boxplot(avg_silhouette_coef)

plt.show()

#### b) Même chose mais avec un nombre d'executions interne plus grand :

Ici nous allons effectuer la même opération qu'au dessus en changeant la valeur du *n_init* par une valeur plus élevée. Pour rappel le paramètre *n_init* représente le nombre de fois où l'algorithme du k-means sera exécuté avec différentes graines pour les centroïdes. Le meilleur résltat en terme d'inertie sera retenue parmi tous les *n_init*.

In [ ]:
avg_silhouette_coef = []
sse_list = []
k = 6
for i in range(600):
    km = KMeans(n_clusters=k, init='random', n_init = 30)
    km.fit(df4)
    labels = km.predict(df4)
    sse_list.append(np.sqrt(km.inertia_))
    avg_silhouette_coef.append(metrics.silhouette_score(df4, labels,metric='euclidean'))

In [ ]:
plt.figure(figsize=(14, 8))

plt.subplot(1,2,1)
plt.hist(sse_list, color='coral', edgecolor = "black")
plt.title("SSE", fontsize = 18)

plt.subplot(1,2,2)
plt.hist(avg_silhouette_coef, color = 'coral', edgecolor = "black")
plt.title("Average silhouette coeff", fontsize = 18)


plt.show()

#### c) Evaluation de la stabilité de convergence un nombre de clusters différents :

In [ ]:
def compute_stability(km,df,iterations=100):
    avg_silhouette_coef = []
    sse_list = []
    for i in range(500):
        km.fit(df)
        labels = km.predict(df)
        avg_silhouette_coef.append(metrics.silhouette_score(df, labels,metric='euclidean'))
    avg_silhouette_coef = np.asarray(avg_silhouette_coef)
    return(avg_silhouette_coef.std())

In [ ]:
stability = []
Ks = range(2,12)
for k in Ks:
    km = KMeans(n_clusters=k,init='random',n_init=5)
    stability.append(compute_stability(km,df4))

In [ ]:
plt.plot(Ks,stability,"o-")
plt.xlabel("number of clusters ",fontsize=20)
plt.ylabel("Avg. Silhouette Coef")

In [ ]:
print("Le nombre de clusters pour une stabilité optimale est : ", int(stability.index(max(stability))) + 2)

## 4.2 Hierachical clustering :

### 4.2.1 Méthode : "complete"

In [ ]:
z_complete = sch.linkage(df4, method = 'complete', metric='euclidean')

In [ ]:
z_complete.shape

In [ ]:
plt.figure(figsize=(20, 40))
dendro = sch.dendrogram(z_complete, 
                        orientation='right', 
                        leaf_rotation=0, 
                        leaf_font_size=14,
                        labels = df4.index)
plt.title("Dendrogram with complete methode")
plt.ylabel("Countries")
plt.xlabel("Distance")
plt.grid(False)
plt.show()

### 4.2.2 Méthode : "single"

In [ ]:
z_single = sch.linkage(df4, method = 'single', metric='euclidean')

In [ ]:
z_single.shape

In [ ]:
plt.figure(figsize=(20, 40))
dendro = sch.dendrogram(z_single, 
                        orientation='right', 
                        leaf_rotation=0, 
                        leaf_font_size=14,
                        labels = df4.index)
plt.title("Dendrogram with single methode")
plt.ylabel("Countries")
plt.xlabel("Distance")
plt.grid(False)
plt.show()

In [ ]:
# 2nd derivative of the distances

acceleration_complete = np.diff(z_complete[:,2], 2)
acceleration_single = np.diff(z_single[:,2], 2) 

In [ ]:
plt.figure(figsize=(32, 18))

plt.subplot(131)
plt.plot(z_single[:, 2], 'o-', color = "blue")
plt.plot(z_complete[:, 2], 'o-', color = "orange")
plt.legend(["single","complete"])
plt.ylabel("Hauteur", fontsize = 22)
plt.xlabel("Pays par ordre croissant", fontsize = 22)
plt.grid(axis='y')

plt.subplot(132)
plt.plot(z_complete[:, 2], 'o-', color = "orange")
plt.plot(acceleration_complete, 'o-', color='green')
plt.ylabel("Hauteur", fontsize = 22)
plt.legend(["complete","dérivée seconde de complete"], fontsize = 22)
plt.xlabel("Pays par ordre croissant", fontsize = 22)

plt.subplot(133)
plt.plot(z_single[:, 2], 'o-', color = "blue")
plt.plot(acceleration_complete, 'o-', color='green')
plt.legend(["single","dérivée seconde de signle"], fontsize = 22)
plt.ylabel("Hauteur", fontsize = 22)
plt.xlabel("Pays par ordre croissant", fontsize = 22)

## 4.3 DB scan :

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN

Une méthode pour estimer le meilleur paramètre **eps** pour notre DBscan serait de calculer la distance de chaque point par rapport à son voisin le plus proche en utilisant les **NearestNeighbors**. Le point lui-même est inclus dans n_neighbors. Nous obtenons ensuite deux tableaux:

- Un qui contient la distance aux points n_neighbors les plus proches ;
- Le deuxième contient l'indice pour chacun de ces points.

In [ ]:
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(df4.values)
distances, indices = nbrs.kneighbors(df4.values)
distances = np.sort(distances, axis=0)
distances = distances[:,1]

Nous pouvons ordonner nos résultats et les afficher dans un plot semblable à ceux déja vu pour des les SSE. 

In [ ]:
plt.figure(figsize=(12,8))
ax = plt.subplot(111)
eps_curve = ax.plot(distances, 'o-', color='b')
plt.ylabel("eps", fontsize = 18)
plt.show() 

Enfin la valeur optimale pour le paramètre **eps** trouve à la courbure maximale de la courbe, lu en ordonnée.

Pour le déterminer nous allons importer la librairie Kneed qui va utiliser la méthode "des genoux".

In [ ]:
from kneed import KneeLocator
points = eps_curve[0].get_data() # on récupère les points de la courbe (x et y)

kneedle = KneeLocator(points[0], points[1], S=1.0, curve="convex", direction="increasing")
knee = int( kneedle.knee )
print("Meilleur paramètre epsilon  : ", round(points[1][knee], 3) )

In [ ]:
dbscan = DBSCAN(eps = 1.478, min_samples=4)
dbscan.fit(df4)

In [ ]:
db_clusters = dbscan.labels_

In [ ]:
print(db_clusters)

In [ ]:
# Contingency table of species vs cluster labels
crosstab = pds.crosstab(db_clusters,classes)
sns.heatmap(crosstab, annot=True)

In [ ]:
# Cluster views in 2D projections

db_results = df4.copy()
db_results['cluster']= db_clusters
classes_map = classes.map({'Africa':0, 'Asia':1, 'Europe':2, 'North America': 3, 'South America':4})
db_results['class']= classes_map.values

In [ ]:
db_results.head()

In [ ]:
sns.pairplot(data=db_results,hue='cluster')
plt.show()

## 4.4 Decision tree :



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
import graphviz

In [ ]:
df_train, df_test, classes_train, classes_test = sklearn.model_selection.train_test_split(df4, df3.Continents,
                                                             test_size=1/3, 
                                                             train_size=2/3,
                                                             random_state=None, 
                                                             shuffle=True, 
                                                             stratify=None)

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(df_train, classes_train)

In [ ]:
classes_predicted = clf.predict(df_test)

In [ ]:
pds.crosstab(classes_test, classes_predicted)

In [ ]:
# Autre façon d'avoir la même table :

from sklearn.metrics import confusion_matrix
confusion_matrix(classes_test, classes_predicted)

In [ ]:
accuracy_score (classes_test, classes_predicted)

In [ ]:
plt.figure(figsize=(18,10))
tree.plot_tree(clf,filled=True,class_names=clf.classes_)
plt.show()

In [ ]:
# Autre visualisation plus sympa utilisant graphviz  (décommenter pour visualier):

tree_plot = tree.export_graphviz(clf, out_file = None, 
                           feature_names = df_train.columns,
                           class_names = clf.classes_, 
                           filled=True, rounded = True,
                           special_characters = True)

graph = graphviz.Source(tree_plot)
#display ( graph ) 

## 4.5 Validation croisée :

In [ ]:
from sklearn.model_selection import StratifiedKFold #cross-validation splitter
from sklearn.model_selection import cross_validate #cross-validation evaluation of metrics

In [ ]:
tests = ['accuracy', 
         'precision_macro', 
         'precision_weighted', 
         'recall_macro', 
         'recall_weighted', 
         'f1_macro', 
         'f1_weighted']

cv = StratifiedKFold(n_splits=5, random_state=10, shuffle=True)
cv_scores = cross_validate(clf, df4, classes, scoring = tests, cv = cv, return_train_score = False)

In [ ]:
pds.DataFrame(cv_scores).mean()